# **Installing packages**

In [ ]:
!pip install transformers sentencepiece tqdm --quiet
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer
import torch
from tqdm import tqdm 

# **Traduction de la colonne "text"**

In [ ]:
#Chargement du dataset poki 
input_path = "/kaggle/input/pokir-dataset/poki_roberta_predictions.csv"
df = pd.read_csv(input_path, encoding="utf-8")

print("Colonnes trouvées :", df.columns.tolist())
print("Aperçu avant traduction :")
print(df.head())

if 'text' not in df.columns:
    raise ValueError("La colonne 'text' n'existe pas dans le dataset !")

#Charger le modèle de traduction
model_name = "Helsinki-NLP/opus-mt-en-ar"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

def translate_batch(texts):
    """
    Traduit une liste de textes EN → AR
    """
    tokens = tokenizer(texts, return_tensors="pt", padding=True, truncation=True).to(device)
    with torch.no_grad():
        gen = model.generate(**tokens)
    arabic_texts = tokenizer.batch_decode(gen, skip_special_tokens=True)
    return arabic_texts

# Traduire uniquement la colonne 'text'
texts = df['text'].fillna("").astype(str).tolist()
translated_cells = []
batch_size = 16

# Boucle avec barre de progression
for i in tqdm(range(0, len(texts), batch_size), desc="Traduction en arabe"):
    batch = texts[i:i+batch_size]
    translated = translate_batch(batch)
    translated_cells.extend(translated)

# Remplacer la colonne 'text' par sa version traduite
df['text'] = translated_cells

# Sauvegarder le résultat
output_path = "pokir_dataset_arabe.csv"
df.to_csv(output_path, index=False, encoding="utf-8")

print(f"Terminé ! La colonne 'text' a été traduite en arabe et sauvegardée sous {output_path}")
